# Brain Tumor Identification
By Jonathan Nunez

abstract

# Business Understanding

# Data Understanding

## Data preparation

In [1]:
# For Google Colab only
# Cell output cleared as it only contains installation progress information
!pip install keras
!pip install keras_preprocessing
!pip install tensorflow
!pip install scikeras[tensorflow]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [2]:
# Importing tools and modules
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
import os


In [3]:
# Code taken from stackoverflow user fdsig
import json
import os
from pathlib import Path

# your api key
api_key = {
'username':"",
'key':""}

# uses pathlib Path
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

# opens file and dumps python dict to json object
with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)

In [4]:
!kaggle datasets download -d ishans24/brain-tumor-dataset
!mkdir /content/brain_tumor_dataset
!unzip brain-tumor-dataset.zip -d /content/brain_tumor_dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/brain_tumor_dataset/meningioma/meningioma515.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma516.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma517.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma518.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma519.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma52.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma520.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma521.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma522.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma523.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma524.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma525.png  
  inflating: /content/brain_tumor_dataset/meningioma/meningioma526.png  
 

In [5]:
data_dir = 'brain_tumor_dataset'

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(256, 256),
    batch_size=32,
    seed=42)

Found 10560 images belonging to 4 classes.


In [8]:
train_generator.class_indices

{'glioma': 0, 'meningioma': 1, 'no_tumor': 2, 'pituitary': 3}

In [ ]:
X_train, y_train, X_test, y_test = train

# Modeling

# Evaluation

# Limitations

# Next Steps